In [2]:
!pip install openai==0.28.1
!pip install fpdf==1.7.2

  Using cached fpdf-1.7.2-py2.py3-none-any.whl


In [3]:
import pandas as pd
import openai
import time
from fpdf import FPDF

openai.api_key = 'KEY_HERE'

## Load the dataset

In [4]:
df = pd.read_csv('https://raw.githubusercontent.com/asocastro/AI_First_Day_4_StockWiz/refs/heads/main/stock.csv')

In [7]:
df.sample(5)

,Date,Season,Product_ID,Units_Sold,Inventory_Level,Cost_Price,Selling_Price,Revenue,Profit_Margin,Discounts_Given
82,2023-08-29,Summer,Samsung Smart TV,352,738,1122.75,1332.61,469078.91,73870.75,41418.83
99,2023-10-13,Fall,PlayStation 5,87,942,1351.08,1762.53,153339.93,35796.05,23908.85
37,2023-04-16,Spring,Samsung Smart TV,174,769,508.31,962.25,167431.91,78986.55,2610.23
86,2023-09-13,Fall,MacBook Air,456,658,892.14,1086.33,495366.47,88550.14,94468.64
35,2023-04-16,Spring,iPhone 14,412,514,731.21,841.77,346810.33,45553.13,52533.24


## Understanding the Dataset

In [10]:
# Get Column Names and Data Types
column_info = df.dtypes
print(column_info, "\n")


Date                object
Season              object
Product_ID          object
Units_Sold           int64
Inventory_Level      int64
Cost_Price         float64
Selling_Price      float64
Revenue            float64
Profit_Margin      float64
Discounts_Given    float64
dtype: object 



In [12]:
# Identify Categorical and Numerical Features
categorical_features = df.select_dtypes(include=['object']).columns
numerical_features = df.select_dtypes(include=['int64','float64']).columns
print('Categorical Features', categorical_features)
print('Numerical Features', numerical_features)



Categorical Features Index(['Date', 'Season', 'Product_ID'], dtype='object')
Numerical Features Index(['Units_Sold', 'Inventory_Level', 'Cost_Price', 'Selling_Price',
       'Revenue', 'Profit_Margin', 'Discounts_Given'],
      dtype='object')


In [15]:
# Get Unique Values for Categorical Features with Limitation
def get_unique_values(col, max_display=10):
  uniques = df[col].unique().tolist()
  if len(uniques) > max_display:
    return uniques[:max_display] + ["..."]

unique_values = {col: get_unique_values(col) for col in categorical_features}

print("Unique Values for Categorical")
for col, uniques in unique_values.items():
    print(f" - {col}: {uniques}")
print()


Unique Values for Categorical
 - Date: ['2023-01-01', '2023-01-16', '2023-01-31', '2023-02-15', '2023-03-02', '2023-03-17', '2023-04-01', '2023-04-16', '2023-05-01', '2023-05-16', '...']
 - Season: None
 - Product_ID: None



In [16]:
# Get Descriptive Statistics for Numerical Features
numerical_stats = df[numerical_features].describe()
print(numerical_stats)



       Units_Sold  Inventory_Level   Cost_Price  Selling_Price        Revenue  \
count   125.00000       125.000000   125.000000     125.000000     125.000000   
mean    278.24000       555.392000   916.947840    1204.693680  332605.931680   
std     129.58553       263.248622   327.658105     336.194736  179638.618546   
min      51.00000       100.000000   300.280000     506.210000   37591.830000   
25%     171.00000       321.000000   669.380000     982.700000  177097.400000   
50%     282.00000       545.000000   930.680000    1185.600000  314538.860000   
75%     384.00000       793.000000  1203.960000    1489.860000  477162.200000   
max     484.00000       983.000000  1498.150000    1827.500000  715154.100000   

       Profit_Margin  Discounts_Given  
count     125.000000       125.000000  
mean    79485.856640     34135.085040  
std     53131.715744     29770.898231  
min      9648.180000       270.500000  
25%     39528.370000     11133.100000  
50%     67889.800000     23528

## Generating the Template

In [18]:
def generate_template(df,
                      column_info,
                      categorical_features,
                      numerical_features,
                      unique_values,
                      numerical_stats):

  # Construct a summary of the dataframe's structure
  column_summary = "Column Names and Data Types:\n"
  for col, dtype in column_info.items():
    column_summary += f" - {col}: {dtype}\n"


  # Unique values for categorical features
  unique_values_str = "Unique Values for Categorical Features:\n"
  for col, uniques in unique_values.items():
    unique_values_str += f" - {col}: {uniques}\n"


  # Descriptive statistics for numerical features
  numerical_stats_str = "Descriptive Statistics for Numerical Features:\n"
  for col in numerical_features:
    numerical_stats_str += f" - {col}:\n"
    for stat_name, value in numerical_stats[col].items():
      numerical_stats_str += f" - {stat_name}: {value}\n"


  # Define the system prompt
  system_prompt = """You are an intelligent assistant that creates descriptive templates for transforming dataframe rows into coherent paragraphs.
    Analyze the provided dataframe structure and generate a template sentence that includes placeholders for each column.
    Ensure the template is contextually relevant and maintains grammatical correctness."""


  # Define the user prompt
  user_prompt = f"""
  Analyze the following dataframe structure and create a descriptive template with placeholders for each column.

  <column_summary>
  {column_summary}
  </column_summary>

  <unique_values>
  {unique_values_str}
  </unique_values>

  <numerical_stats>
  {numerical_stats_str}
  </numerical_stats>

  Use the exact column names from the column_summary in generating the variable names in the template,
  as they will be populated with the actual values in the dataset.

  Example Template about a Spotify dataset:
  "{{artist}} gained {{streams}} streams in the song '{{song}}' that was a hit in {{date}}."


  Output only the template without any explanation or introduction.
  The template's variables will be dynamically replaced so make sure they're formatted properly."""


  # Generate the template (with retries)
  retries = 3
  for attempt in range(retries):
      try:
          response = openai.ChatCompletion.create(
              model="gpt-4o-mini",
              temperature=0.3,
              max_tokens=1024,
              messages=[
                  {"role": "system", "content": system_prompt},
                  {"role": "user", "content": user_prompt}
              ]
          )
          template = response['choices'][0]['message']['content'].strip()
          return template
      except Exception as e:
          print(f"Error generating template (Attempt {attempt + 1}/{retries}): {e}")
          time.sleep(2)  # Wait before retrying

  return None


In [19]:
# Generate the template
template = generate_template(df,
                             column_info,
                             categorical_features,
                             numerical_features,
                             unique_values,
                             numerical_stats)
print(template)


"On {Date}, during the {Season}, the product with ID {Product_ID} achieved sales of {Units_Sold} units, resulting in an inventory level of {Inventory_Level}. The cost price for this product was {Cost_Price}, while it was sold at a price of {Selling_Price}. This led to a total revenue of {Revenue} and a profit margin of {Profit_Margin}. Additionally, discounts given amounted to {Discounts_Given}."


## Populating the Template with Actual Values

In [20]:
def populate_template(template, row):

  # Convert row to dictionary and replace NaN with 'N/A'
  row_dict = row.to_dict()
  for key, value in row_dict.items():
    if pd.isna(value):
      row_dict[key] = 'N/A'
  paragraph = template.format(**row_dict)

  return paragraph

  # Generate the populated template per row



In [21]:
# Apply the populate_template function to each row in the dataframe
df['paragraph'] = df.apply(lambda row: populate_template(template, row), axis=1)


In [22]:
df['paragraph'][0]

'"On 2023-01-01, during the Winter, the product with ID iPhone 14 achieved sales of 199 units, resulting in an inventory level of 156. The cost price for this product was 1134.53, while it was sold at a price of 1580.6. This led to a total revenue of 314538.86 and a profit margin of 88767.06. Additionally, discounts given amounted to 12395.85."'

## Saving the Populated Templates (Natural Language Texts)

In [25]:
def save_content_to_txt(paragraphs, txt_filename):
    try:
        with open(txt_filename, 'w', encoding='utf-8') as file:

            for para in paragraphs:
                file.write(para + '\n\n')  # Double newline for separation

        print(f"All content has been saved as '{txt_filename}'.\n")
    except Exception as e:
        print(f"Error saving content to text file: {e}")

In [26]:
def save_content_to_pdf(paragraphs, pdf_filename):
    try:
        pdf = FPDF()
        pdf.add_page()
        pdf.set_auto_page_break(auto=True, margin=15)
        pdf.set_font("Arial", size=12)

        # Add each paragraph
        for para in paragraphs:
            pdf.multi_cell(0, 10, para)
            pdf.ln()  # Add a line break between paragraphs

        pdf.output(pdf_filename)
        print(f"All content has been saved as '{pdf_filename}'.\n")
    except Exception as e:
        print(f"Error saving content to PDF: {e}")

In [28]:
# Save the paragraphs to both text and PDF files
save_content_to_txt(df['paragraph'].to_list(),
                    txt_filename='nlg.txt')
save_content_to_pdf(df['paragraph'].to_list(),
                    pdf_filename='nlg.pdf')


All content has been saved as 'nlg.txt'.

All content has been saved as 'nlg.pdf'.

